In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [27]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [28]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [31]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items_weights.loc[:, 'weight'] = items_weights.loc[:, 'sales_value'] / items_weights.loc[:, 'sales_value'].sum()
    
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [35]:
%%time

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(data_test[['item_id', 'sales_value']], n=5))
result

Wall time: 8.22 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[897083, 1095100, 9420314, 997987, 1067795]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[835619, 1128182, 7157442, 901666, 898847]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[17381995, 1079913, 1075032, 1043084, 891632]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[943940, 1070803, 1087174, 6534178, 1025176]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[9420067, 878222, 1029743, 12301839, 835098]"
...,...,...,...
2037,2496,[6534178],"[9881904, 997200, 984573, 6534178, 1049897]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[942189, 1123842, 1028290, 6534178, 6534178]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1044078, 835618, 5569374, 894627, 893018]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[15716283, 6533889, 10311570, 827047, 8249140]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [37]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [38]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list 
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)    
    
    return precision

In [ ]:
precision_at_k(convert(result.loc[0, column]), convert(row['actual']))

In [70]:
def convert(list_str):
    lst = list_str[2:-1].split(', ')
    return [int(el.strip()) for el in lst]

def convert_2(list_str):
    lst = list_str[2:-1].split()
    return [int(el.strip()) for el in lst]

test_precision_5 = {'algorithm': result.columns[2:],
                   'values': []}

for column in result.columns[2:]:
    
    test_precision_5['values'].append(result.apply(lambda row: precision_at_k(convert(row[column]), convert_2(row['actual'])), 
                                              axis=1).mean())

test_precision_5 = pd.DataFrame(test_precision_5)
test_precision_5

,algorithm,values
0,random_recommendation,0.000490
1,popular_recommendation,0.116944
2,itemitem,0.025759
3,cosine,0.026249
4,tfidf,0.027424
5,own_purchases,0.178616


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code